In [251]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [252]:
from lego import *

In [259]:
a = Lambda(lambda x: 1)

Add([a,a,a,a])(0)

4

In [256]:
def resnet_basic_block(in_features, out_features):
    return nn.Sequential(
        Add([nn.Sequential(
        conv3x3_bn_act(in_features, in_features, stride=2, bias=False),
        conv3x3_bn(in_features, out_features, bias=False))], 
            shortcut=nn.Sequential(conv3x3_bn(in_features, out_features, stride=2, bias=False))),
        nn.ReLU())

ResNet34 = nn.Sequential(
    Conv2dPad(3, 64, kernel_size=7, stride=4),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, padding=1),
    resnet_basic_block(64, 128),
    resnet_basic_block(128, 256),
    resnet_basic_block(256, 512),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(512, 1000)
)

In [257]:
x = torch.rand((1,3,224,244))

ResNet34(x).shape

torch.Size([1, 1000])

In [173]:
class ResidualWithShortcut(Residual):
    def __init__(self, blocks, shortcut, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.blocks = blocks
        self.shortcut = shortcut  
    
    def forward(self, x):
        if self.should_apply_shortcut: x = self.shortcut(x)
        super().forward(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [172]:
ResidualBlock(a, mode='add')(torch.tensor(2))

TypeError: __init__() got an unexpected keyword argument 'mode'

In [154]:
cat = Cat(nn.ModuleList([conv3x3(3, 64)]), nn.ModuleList([conv3x3(64*2, 3)]))

# cat(torch.rand((1,3, 224, 224))).shape

cat

Concat(
	Conv2dPad(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) -> Conv2dPad(128, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [117]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

### Unet

In [43]:
down = lambda in_features, out_features: nn.Sequential(
    conv3x3(in_features, in_features),
    conv3x3(in_features, out_features),
    nn.MaxPool2d(out_features)
)

up = lambda in_features, out_features : nn.Sequential(
    conv3x3(in_features, in_features),
    conv3x3(in_features, out_features),
    nn.ConvTranspose2d(out_features, out_features, kernel_size=2)
)

Concat(
    [
        down(64, 128),
        down(128, 256),
        down(256, 512),
        down(512, 1024),
    ],
    [
        up(1024, 512 * 2),
        up(256 * 2, 128),
        up(128 * 2, 64),
        nn.Sequential(
            conv3x3(64), 
            conv3x3(64))
    ]
)